# Probabilidad - Ejercicio 2

Esta tarea consiste en analizar el dataset 'credit.csv', que contiene un conjunto de datos sobre impagos en créditos bancarios. El dataset contiene una serie de variables numéricos, otras categóricas y la variable impago ('default').

Sobre este dataset tenéis que calcular:

- Probabilidad de impago, P(default=2)
- Probabilidad de impago condicionado a la variable 'housing' (considerad todos los valores de la variable)
- Probabilidad de impago condicionado a las variables 'dependents'=2 y 'telephone'='none'
- Probabilidad de impago condicionado a las variables 'dependents'=1, 'telephone'='yes' y 'personal_status'='female'

Además, debéis comprobar si las variables 'housing' y 'property' son independientes.

Debéis realizar la tarea en un Notebook de Jupyter, que debéis entregar ejecutado y sin errores.

# Resolución
## Carga y exploración del dataset
Lo primero que hacemos es importar los paquetes que utilzaremos y cargar el dataset que se nos proporciona. Como es un dataset con muchas columnas podemos ver sus nombres accediendo al atributo correspondiente del dataframe. Para ver la cantidad de valores diferentes que puede tomar la variable ```default``` podemos utilizar la función ```pd.unique```.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
df = pd.read_csv("credit.csv")
df.head()

,checking_balance,months_loan_duration,credit_history,purpose,amount,savings_balance,employment_length,installment_rate,personal_status,other_debtors,...,property,age,installment_plan,housing,existing_credits,default,dependents,telephone,foreign_worker,job
0,< 0 DM,6,critical,radio/tv,1169,unknown,> 7 yrs,4,single male,none,...,real estate,67,none,own,2,1,1,yes,yes,skilled employee
1,1 - 200 DM,48,repaid,radio/tv,5951,< 100 DM,1 - 4 yrs,2,female,none,...,real estate,22,none,own,1,2,1,none,yes,skilled employee
2,unknown,12,critical,education,2096,< 100 DM,4 - 7 yrs,2,single male,none,...,real estate,49,none,own,1,1,2,none,yes,unskilled resident
3,< 0 DM,42,repaid,furniture,7882,< 100 DM,4 - 7 yrs,2,single male,guarantor,...,building society savings,45,none,for free,1,1,2,none,yes,skilled employee
4,< 0 DM,24,delayed,car (new),4870,< 100 DM,1 - 4 yrs,3,single male,none,...,unknown/none,53,none,for free,2,2,2,none,yes,skilled employee


In [3]:
df.columns

Index(['checking_balance', 'months_loan_duration', 'credit_history', 'purpose',
       'amount', 'savings_balance', 'employment_length', 'installment_rate',
       'personal_status', 'other_debtors', 'residence_history', 'property',
       'age', 'installment_plan', 'housing', 'existing_credits', 'default',
       'dependents', 'telephone', 'foreign_worker', 'job'],
      dtype='object')

In [4]:
pd.unique(df['default'])

array([1, 2], dtype=int64)

### Probabilidad de impago, P(default=2)

In [5]:
df_impago = pd.crosstab(df["default"], "count")
df_impago

col_0,count
default,
1,700
2,300


In [6]:
prob_impago = df_impago.loc[2].sum() / df_impago.sum().sum()
print(f"La probabilidad de impago es: {prob_impago}")

La probabilidad de impago es: 0.3


### Probabilidad de impago condicionado a la variable 'housing' (considerad todos los valores de la variable)

In [7]:
df_impago_housing = pd.crosstab(df["default"], df["housing"])
df_impago_housing

housing,for free,own,rent
default,,,
1,64,527,109
2,44,186,70


A partir de la expresión de la probabilidad condicionada podemos escribirla para este caso particular: 
$$ P(A|B) = \frac{P(A \cap B)}{P(B)} \rightarrow P(default = 2|housing) = \frac{P(default = 2 \cap housing)}{P(housing)}$$
Para obtener las $P(default = 2 \cap housing)$ solamente tenemos que mirar la tabla y dividir por el número total de elementos, y para $P(housing)$ sumamos las columnas y divimos entre el total de elementos.

Como podemos observar, en ambos casos dividimos por el total de elementos, por lo que podríamos mejorar el proceso si normlizasemos la tabla anterior al total de elementos y así tendríamos directamente las probabilidad, pero en este ejercicio lo vamos a dejar así para familizarizarnos con el cálculo de las probabilidades.

In [8]:
p_impago_housing = [(df_impago_housing.loc[2,housing] / df_impago_housing.sum().sum()) / (df_impago_housing[housing].sum() / df_impago_housing.sum().sum()) for housing in df_impago_housing.columns]
for i, housing in enumerate(df_impago_housing.columns):
    print(f"La probabilidad de impago condicionado a {housing} es: {p_impago_housing[i]:.2f}")

La probabilidad de impago condicionado a for free es: 0.41
La probabilidad de impago condicionado a own es: 0.26
La probabilidad de impago condicionado a rent es: 0.39


In [9]:
df_impago_dep_tel = pd.crosstab(df["default"], [df["dependents"], df["telephone"]])
df_impago_dep_tel

dependents    1         2    
telephone  none  yes none yes
default                      
1           349  242   60  49
2           152  102   35  11

### Probabilidad de impago condicionado a las variables 'dependents'=2 y 'telephone'='none'

In [10]:
p_imp_2_none = df_impago_dep_tel.loc[2][(2, "none")] / df_impago_dep_tel.sum().sum()
p_2_none = df_impago_dep_tel[(2, "none")].sum() / df_impago_dep_tel.sum().sum()

p_imp_cond_2_none = p_imp_2_none / p_2_none
print(f"La probabilidad de impago condicionada a dependentes = 2 y telephone = 'none' es: {p_imp_cond_2_none:.2f}")

La probabilidad de impago condicionada a dependentes = 2 y telephone = 'none' es: 0.37


### Probabilidad de impago condicionado a las variables 'dependents'=1, 'telephone'='yes' y 'personal_status'='female'

In [11]:
df_impago_dep_tel_ps = pd.crosstab(df["default"], [df["dependents"], df["telephone"], df["personal_status"]])
df_impago_dep_tel_ps

dependents                  1                                                \
telephone                none                                           yes   
personal_status divorced male female married male single male divorced male   
default                                                                       
1                          14    123           43         169            13   
2                          12     69           15          56             8   

dependents                                                  2         \
telephone                                                none          
personal_status female married male single male divorced male female   
default                                                                
1                   70           23         136             2      5   
2                   34            9          51             0      5   

dependents                                                                  \
telephone                                          yes                       
personal_status married male single male divorced male female married male   
default                                                                      
1                          0          53             1      3            1   
2                          1          29             0      1            0   

dependents                   
telephone                    
personal_status single male  
default                      
1                        44  
2                        10

In [12]:
p_imp_1_yes_fem = df_impago_dep_tel_ps.loc[2][(1, 'yes', 'female')] / df_impago_dep_tel_ps.sum().sum()
p_1_yes_fem = df_impago_dep_tel_ps[(1, 'yes', 'female')].sum() / df_impago_dep_tel_ps.sum().sum()

p_imp_cond_1_yes_fem = p_imp_1_yes_fem / p_1_yes_fem
print(f"La probabilidad de impago condicionada a dependentes = 1, telephone = 'yes' y personal_status = 'female' es: {p_imp_cond_1_yes_fem:.2f}")

La probabilidad de impago condicionada a dependentes = 1, telephone = 'yes' y personal_status = 'female' es: 0.33


### Comprobar si las variables 'housing' y 'property' son independientes

Sabemos que dos variables son independientes cuándo la probabilidad conjunta es igual al producto de las probabilidades de cada uno de ellos. Matemáticamente esto se expresa como:
$$ P(A \cap B) = P(A)P(B) $$
Introduciendo esta expresión en la ecuación de la probabilidad condicionada:
$$ P(A|B) = \frac{P(A \cap B)}{P(B)} = \frac{P(A)P(B)}{P(B)} = P(A) $$
Partiendo de esto, si queremos comprobar si las variabes *housing* y *property* son independientes, podemos calcular su probablidad y comprobar si se cumple la igualdad anterior.

In [13]:
df_housing_property = pd.crosstab(df["housing"], df["property"])
df_housing_property

property,building society savings,other,real estate,unknown/none
housing,,,,
for free,2,1,1,104
own,184,271,226,32
rent,46,60,55,18


In [14]:
p_own = len(df[df["housing"] == "own"]) / len(df)
p_realestate = len(df[df["property"] == "real estate"]) / len(df)

print(f"La probabilidad de housing = own es: {p_own}.\nLa probabilidad de property = real estate es: {p_realestate}.")

La probabilidad de housing = own es: 0.713.
La probabilidad de property = real estate es: 0.282.


In [15]:
p_own = df_housing_property.loc["own"].sum() / df_housing_property.sum().sum()
p_own_re = df_housing_property.loc["own"]["real estate"] / df_housing_property.sum().sum()
p_re = df_housing_property["real estate"].sum() / df_housing_property.sum().sum()

p_own_cond_re = p_own_re / p_re

print(f"La probabilidad de own condicionada a real estate es: {p_own_cond_re:.2f}")
print(f"Mientras que la probabilidad de own es: {p_own:.2f}")

if p_own == p_own_cond_re:
    print("Por lo que se cumple la igualdad y son variables independientes.")
else:
    print("Por lo que no se cumple la igualdad y no son variables independientes.")

La probabilidad de own condicionada a real estate es: 0.80
Mientras que la probabilidad de own es: 0.71
Por lo que no se cumple la igualdad y no son variables independientes.


En este caso hemos decidido comprobarlo utilizando "onw" y "real state" pero el resultado sería el mismo independientemente del 'housing' y el 'property' elegidos.